In [1]:
%load_ext autoreload
%autoreload 2

import re
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 12, 6
matplotlib.rcParams['font.size'] = 18  # Probably OS Dependent

In [3]:
data_files = glob('../public/*.json')

dfs = []
for file_ in data_files:
    df = pd.read_json(file_)
    df['state'] = re.search('public/(.+).json', file_).group(1).title()
    dfs += [df]
    
df = pd.concat(dfs).reset_index()

def empty_null(el_list):
    if type(el_list) != list:
        return []
    return el_list
df['emails'] = df['emails'].apply(empty_null)
df['faxes'] = df['faxes'].apply(empty_null)
df['phones'] = df['phones'].apply(empty_null)

df.sample(5, random_state=42)

,index,locale,official,emails,faxes,phones,county,address,party,state,contacts,physicalAddress,city,url,faxs,urls,officla
2517,412,South Bristol,Annette Sue Farrin,[],[(207) 563-1798],[(207) 563-3977],NaN,"[470 Clarks Cove Rd - Walpole, , South Bristol...",NaN,Maine,NaN,NaN,South Bristol,NaN,NaN,NaN,NaN
4087,1321,Merritt Township:Bay County,Kathy Parrent,[merrittclerk@yahoo.com],[(989) 659-3475],[(989) 659-2136],Bay County,NaN,NaN,Michigan,NaN,NaN,Merritt Township,NaN,NaN,NaN,NaN
3127,361,South Haven Township:Van Buren County,Brenda Bertorelli,[bbertorelli@southhaventwp.com],[(269) 637-6250],[(269) 637-3305],Van Buren County,NaN,NaN,Michigan,NaN,NaN,South Haven Township,NaN,NaN,NaN,NaN
151,58,Brantley County,Christine Turner,[voterreg@btconline.net],[(912) 462-6197],[],Brantley County,NaN,NaN,Georgia,{'Mailing Address': {'kind': 'Mailing Address'...,NaN,NaN,NaN,NaN,NaN,NaN
1945,1693,Town of West Bend:Washington County,Julie Ihlenfeld,[clerk@townofwestbend.com],[(262)338-0427],[],Washington County,"6355 County Road Z, West Bend, Wi 53095-9201",NaN,Wisconsin,NaN,"6355 County Road Z, West Bend, Wi 53095-9201",Town of West Bend,NaN,NaN,NaN,NaN


In [4]:
def non_empty_element(el_list):
    values = [len(x.strip()) > 0 for x in el_list if x]
    if len(values) == 0:
        return False
    return any(values)

def not_null_empty(series):
    return (series.notnull() & series.apply(lambda x: x != ''))

email = df['emails'].apply(empty_null).apply(non_empty_element)
fax = df['faxes'].apply(empty_null).apply(non_empty_element)
phone = df['phones'].apply(empty_null).apply(non_empty_element)
city = not_null_empty(df['city'])
county = not_null_empty(df['county'])

df_valid = pd.DataFrame({
    'official': df['official'].notnull(),
    'locale': not_null_empty(df['locale']),
    'city': city,
    'county': county,
    'city_county': (city | county),
    'email': email,
    'faxes': fax,
    'phones': phone,
    'fax_email': (email | fax),
    'url': df['url'].notnull()
})

df_stat = df_valid.groupby(df['state']).mean()
df_stat.loc[:, 'records'] = df.groupby('state').size()

df_len = df[['state']]
df_len.loc[:, 'emails'] = df['emails'].apply(lambda x: len(x))
df_len.loc[:, 'faxes'] = df['faxes'].apply(lambda x: len(x))
df_len.loc[:, 'phones'] = df['phones'].apply(lambda x: len(x))
df_stat.loc[:, 'email_max'] = df_len.groupby('state')['emails'].max()
df_stat.loc[:, 'fax_max'] = df_len.groupby('state')['faxes'].max()
df_stat.loc[:, 'phone_max'] = df_len.groupby('state')['phones'].max()
df_stat.sort_index()

/Users/tianhui/projects/elections-officials/venv/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/tianhui/projects/elections-officials/venv/lib/python3.7/site-packages/pandas/core/indexing.py:964: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,official,locale,city,county,city_county,email,faxes,phones,fax_email,url,records,email_max,fax_max,phone_max
state,,,,,,,,,,,,,,
Florida,1.000000,1.0,0.000000,1.000000,1.0,1.000000,0.000000,0.000000,1.000000,1.000000,67,1,0,0
Georgia,1.000000,1.0,0.000000,1.000000,1.0,0.968553,0.987421,0.000000,0.993711,0.000000,159,1,1,0
Maine,1.000000,1.0,1.000000,0.000000,1.0,0.000000,0.942460,1.000000,0.942460,0.000000,504,0,1,1
Maryland,0.916667,1.0,0.000000,1.000000,1.0,1.000000,0.000000,0.916667,1.000000,1.000000,24,4,0,2
Michigan,1.000000,1.0,1.000000,1.000000,1.0,0.981107,0.687296,1.000000,0.988925,0.000000,1535,1,1,1
Minnesota,1.000000,1.0,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,87,1,1,1
Nebraska,1.000000,1.0,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,93,1,1,1
Nevada,0.000000,1.0,0.058824,0.941176,1.0,1.000000,0.000000,0.000000,1.000000,1.000000,17,2,0,0
Virginia,1.000000,1.0,0.285714,0.714286,1.0,1.000000,1.000000,0.000000,1.000000,0.541353,133,1,1,0


## Michigan

In [5]:
df_mi = df[df['state'] == 'Michigan']
df_mi['city'].value_counts().head()

Grant Township       11
Sherman Township      9
Lincoln Township      7
Lake Township         6
Garfield Township     6
Name: city, dtype: int64

However, locales are unique

In [6]:
assert((df_mi['city'] + ':' + df_mi['county'].fillna('') == df_mi['locale']).all())
df_mi['locale'].value_counts().head()

Lathrup Village City:Oakland County      1
McKinley Township:Emmet County           1
Shelby Charter Township:Macomb County    1
Norwich Township:Missaukee County        1
Sodus Township:Berrien County            1
Name: locale, dtype: int64

## Wisconsin Unique Identifier

Wisonsin towns cannot be uniquely identified by city

In [7]:
df_wi = df[df['state'] == 'Wisconsin']
df_wi['city'].value_counts().head()

Town of Lincoln       12
Town of Washington     8
Town of Union          7
Town of Scott          7
Town of Grant          6
Name: city, dtype: int64

The locales are unique, but don't properly handle multi-county cities (blank entry)

In [8]:
assert((df_wi['city'] + ':' + df_wi['county'].fillna('') == df_wi['locale']).all())
df_wi['locale'].value_counts().head()

Town of Brooklyn:Washburn County        1
Town of Langlade:Langlade County        1
Town of Ridgeway:Iowa County            1
Village of Somerset:St. Croix County    1
Town of Fairchild:Eau Claire County     1
Name: locale, dtype: int64

So we need to ensure that all multi-county cities are unique across the entire state

In [9]:
multi_county_cities = df_wi.loc[df_wi['county'].isnull(), 'city']
df_wi.loc[df_wi['city'].isin(multi_county_cities), 'city'].value_counts().head()

Village of Wrightstown    1
City of Hartford          1
City of Kiel              1
City of Stanley           1
City of Colby             1
Name: city, dtype: int64